In [1]:
# Build a sample vectorDB
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, Docx2txtLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.retrievers  import BM25Retriever
from langchain.retrievers import EnsembleRetriever

from pathlib import Path
import shutil

# Load blog post
# Load docs

doc_paths = [
    "docs/가스계통_운영규정.pdf",
    "docs/여비규정.pdf",
    "docs/취업규칙.pdf",
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    print("doc_file", doc_file)
    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Split docs

text_splitter = SemanticChunker(
    # chunk_size=1000,
    # chunk_overlap=300,
    OpenAIEmbeddings()
)

document_chunks = text_splitter.split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


doc_file docs/가스계통_운영규정.pdf
doc_file docs/여비규정.pdf
doc_file docs/취업규칙.pdf


In [2]:
# Tokenize and load the documents to the vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

# BM25Retriever
bm25_retriever = BM25Retriever.from_documents(document_chunks)

bm25_retriever.k = 5


# Chroma Retriever
chroma_retriever = vector_db.as_retriever(search_kwargs={'k':5})

# Ensemable Retriever
ensemble_retriever = EnsembleRetriever(
    # retrievers=[bm25_retriever, chroma_retriever, multi_query_retriever],
    # weights=[0.2, 0.4, 0.4]

    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.5, 0.5]        

)

# Create MultiQueryRetriever
llm = ChatOpenAI(temperature=0)

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=ensemble_retriever,
    llm=llm
)


In [7]:
prompt_template = """
당신은 한국가스공사의 규정 전문가입니다. 주어진 질문에 대해 관련 규정을 정확히 인용하여 답변해야 합니다.

질문: {question}

관련 문서의 내용:
{context}

위의 정보를 바탕으로 다음 형식에 맞춰 답변해주세요:
1. 먼저 "네," 또는 "죄송합니다,"로 시작하세요.
2. 관련 규정을 정확히 인용하세요
3. 인용한 규정의 내용을 설명하세요.
4. 필요한 경우 추가 정보나 설명을 제공하세요.

답변:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


# 질문 설정
question = "일이 너무 많아서 연차휴가를 할당된 만큼 다 쓰지 못할거 같아. 그럼 남는 연차휴가는 어떻게 되지? 남는 연차 휴가에 대해 돈으로 받을수 있어?"

# RetrievalQA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=multi_query_retriever,
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT}
)

# 답변 얻기
result = qa_chain({"query": question})

# 답변 및 출처 출력
print("Answer:", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f"- {doc.metadata.get('source', 'Unknown source')}")
    print(f"  Content: {doc.page_content[:200]}...")  # 각 출처의 첫 200자 출력

Answer: 네, 공사의 규정에 따르면 연차휴가는 직원의 자유의사에 따라 적치하여 계산기간 만료 익일부터 1년 이내에 사용해야 합니다. 그러나 연차휴가 중 12일 이내에 근로기준법 제61조에 따라 사용촉진조치를 시행하며, 그 미사용 휴가는 연간 10일 한도로 5년간 저축하여 사용할 수 있습니다. 그러나 5년 이내 또는 퇴직 시까지 사용하지 않은 저축 연차휴가는 자동소멸됩니다. 또한, 공사의 형편에 따라 연차휴가를 사용할 수 없거나 적치하지 않은 경우에는 보수규정이 정하는 바에 따라 수당을 지급합니다. (참조: 제29조)

Sources:
- docs/취업규칙.pdf
  Content: 제28조(특수근무자의 휴일) 교대근무자는 제27조의 규정에 불구하고 교대근무에 따른 휴무일을 휴일로 한다. 제29조(연차유급휴가 )
① 직원이 1년간 8할 이상 출근하였을 때에는 15일의 연차유급휴가를 부여한다 . ② 3년 이상 계속하여 근로한 직원에 대하여는 기본 연차유급 휴가에 계속근로연수 매2년에 대하여 1일을 가산한 유급휴
가를 부여하며 , 이 경우...
- docs/취업규칙.pdf
  Content: 9. 1.> 
제34조(휴가기간의 초과) 제29조 및 제30조에서 정한 휴가일수를 초과한 경우에는 결근으로 본다. <신설 1997. 9. 1.> 
제35조(출근명령 ) 업무상 필요할 때에는 휴일 또는 휴가 중이라도 출근을 명할 수 있다. 제36조(대휴 또는 휴일의 대체)
① 직원이 제35조의 규정에 의하여 휴일 또는 휴가 중에 근무한 때에는 업무에 지장이 ...
- docs/취업규칙.pdf
  Content: 그 기간을 가산한 기간 이내로 하며 분할사용이 가능하고 , 1회 신청기간은 3개월 이상이 되어야 한다. 단축 후 근로시간은 
주당 15시간 이상 35시간 이내로 한다. <신설 2014.1.1, 2024.01.31>
제45조(휴직기간 ) 휴직기간은 다음과 같다. 1....
- docs/취업규칙.pdf
  Content: <삭제, 2024.01.31>
16. 5년 이상 

In [12]:
def _get_context_retriever_chain(vector_db, ensemble_retriever, llm):

    multi_query_retriever = MultiQueryRetriever.from_llm(
        retriever=ensemble_retriever,
        llm=llm
    )


    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, ensemble_retriever, prompt)

    return retriever_chain

In [13]:
def get_conversational_rag_chain(llm, retriever):
    retriever_chain = _get_context_retriever_chain(vector_db, retriever, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are an assistant designed specifically for answering queries based on company regulations. Always respond strictly according to the company's internal regulations, ensuring your answers are aligned with these rules. 
        When providing an answer, first cite the most relevant regulation in detail, including chapter and section numbers if applicable. If multiple regulations apply, list all relevant ones before giving your response. 
        Your goal is to provide the user with clear guidance based on the regulations, so be as specific as possible with the details of the rules and regulations before proceeding with the final answer.
        If no regulation directly applies, inform the user and give your best guidance based on your knowledge of the company's practices.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [14]:
llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0,
    streaming=True,
)

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "일이 너무 많아서 연차휴가를 할당된 만큼 다 쓰지 못할거 같아. 그럼 남는 연차휴가는 어떻게 되지? 남는 연차 휴가에 대해 돈으로 받을수 있어?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream_openai, multi_query_retriever)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

제29조(연차유급휴가)와 관련된 규정을 살펴보면, 다음과 같은 내용이 있습니다:

- 제29조 제5항에 따르면, 연차휴가는 직원의 자유의사에 따라 적치하여 계산기간 만료 익일부터 1년 이내에 사용해야 합니다. 사용하지 못한 연차휴가는 연간 10일 한도로 5년간 저축하여 사용할 수 있으나, 5년 이내 또는 퇴직 시까지 사용하지 않은 저축 연차휴가는 자동소멸됩니다.
- 제29조 제6항에 따르면, 공사의 형편에 따라 연차휴가를 사용할 수 없거나 적치하지 아니한 때에는 보수규정이 정하는 바에 따라 수당을 지급할 수 있습니다.

따라서, 남은 연차휴가는 일정 조건 하에 수당으로 받을 수 있습니다. 다만, 이는 공사의 형편에 따라 결정되므로, 구체적인 상황에 대해서는 소속 부서나 인사팀에 문의하여 확인하는 것이 좋습니다.

죄송합니다, 한국가스공사의 규정에 따르면 연차휴가는 일정한 기간 이내에 사용해야 합니다. 남는 연차휴가는 보통 다음 연도로 이월되거나 사용할 수 있는 기간이 제한될 수 있습니다. 

한국가스공사의 규정 중 제29조(연차유급휴가)에는 연차휴가 사용에 대한 규정이 있습니다. 해당 규정에 따르면 연차휴가는 직원의 자유의사에 따라 적치하여 계산기간 만료 익일부터 1년 이내에 사용해야 합니다. 또한, 공사가 제29조 제1항, 제3항 및 제4항에 따른 연차유급휴가 중 12일의 범위 내에 근로기준법 제61조에 따라 사용촉진조치를 시행하며 그 미사용 휴가는 연간 10일 한도로 5년간 저축 사용할 수 있지만 5년 이내 또는 퇴직 시까지 사용하지 않은 저축 연차휴가는 자동소멸된다고 명시되어 있습니다.

따라서, 남는 연차휴가에 대해 돈으로 받는 것은 규정에 따라 허용되지 않습니다. 연차휴가는 휴가로 사용하거나 일정 기간 이내에 소멸될 수 있습니다. 부가적인 정보가 필요하시면 언제든지 물어보세요.